In [1]:
import warnings
warnings.filterwarnings('ignore')

from numba.core.errors import NumbaWarning, NumbaDeprecationWarning, NumbaPendingDeprecationWarning
warnings.simplefilter('ignore', category=NumbaWarning)
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

In [2]:
import spikeinterface as si
import spikeinterface.extractors as se 
import spikeinterface.postprocessing as spost
import spikeinterface.exporters as sexp
import spikeinterface.widgets as sw
import spikeinterface.curation as scur
import sortingview.views as vv

import numpy as np
import kachery_cloud as kcl

%matplotlib widget
print(f"SpikeInterface version: {si.__version__}")

/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/tridesclous/numba_tools.py:11: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)
/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/tridesclous/numba_tools.py:33: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)
/Users/jcheng/miniconda3/envs/si_env/lib

SpikeInterface version: 0.97.0


Read in recording and sorting files

In [3]:
dirpath = '/Users/jcheng/Documents/Data/20181105/mountains/channel019/'
recording_dir = dirpath + 'dataset'
sorting_file = dirpath + 'output/firings.mda'

In [4]:
recording = se.read_mda_recording(recording_dir, raw_fname='raw_data.mda',)
sorting = se.read_mda_sorting(sorting_file, 30_000)

recording.annotate(is_filtered=True)
recording

MdaRecordingExtractor: 1 channels - 1 segments - 30.0kHz - 5496.020s

Extract waveforms

In [5]:
job_kwargs = dict(n_jobs=10, chunk_duration='1s', progress_bar=True,)
we = si.extract_waveforms(recording, sorting, folder=dirpath+"waveforms", 
                          load_if_exists=False, overwrite=True,
                          max_spikes_per_unit=500,
                          ms_before=1.5, ms_after=2.5,
                          sparse=True,**job_kwargs)

/var/folders/x9/1nq85w8s11l0sv1yqg3364200000h2/T/ipykernel_80365/172480338.py:2: DeprecationWarning: load_if_exists=True/false is deprcated. Use load_waveforms() instead.
  we = si.extract_waveforms(recording, sorting, folder=dirpath+"waveforms",


extract waveforms shared_memory:   0%|          | 0/5497 [00:00<?, ?it/s]

/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/tridesclous/numba_tools.py:11: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)
/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/tridesclous/numba_tools.py:33: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)
/Users/jcheng/miniconda3/envs/si_env/lib

Setting 'return_scaled' to False


extract waveforms memmap:   0%|          | 0/5497 [00:00<?, ?it/s]

/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/tridesclous/numba_tools.py:11: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)
/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/tridesclous/numba_tools.py:33: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)
/Users/jcheng/miniconda3/envs/si_env/lib

Compute sorting metrics

In [6]:
# Spike amplitudes
amplitudes = spost.compute_spike_amplitudes(we, outputs="by_unit", load_if_exists=True, 
                                            **job_kwargs)
# ISI histograms
isi_histograms, hist_bins = spost.compute_isi_histograms(we)

# Correlograms
ccgs, cor_bins = spost.compute_correlograms(we)

# PCA
pc = spost.compute_principal_components(we, n_components=3,
                                        load_if_exists=False,
                                        n_jobs=job_kwargs["n_jobs"], 
                                        progress_bar=job_kwargs["progress_bar"])
# Unit locations
unit_locs = spost.compute_unit_locations(we)

# Template metrics
metrics = spost.compute_template_metrics(we, metric_names=spost.get_template_metric_names())

# Template imilarity
similarity = spost.compute_template_similarity(we)


Setting 'return_scaled' to False


extract amplitudes:   0%|          | 0/5497 [00:00<?, ?it/s]

/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or l

Fitting PCA:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or l

Projecting waveforms:   0%|          | 0/5 [00:00<?, ?it/s]

Generate GUI for viewing and curation of sorting results

In [15]:
kcl.use_sandbox()
we_ts = sw.plot_timeseries(recording, backend="sortingview", mode='map')
we_ss = sw.plot_sorting_summary(we, curation=True, backend='sortingview')

/Users/jcheng/miniconda3/envs/si_env/lib/python3.9/site-packages/spikeinterface/widgets/sortingview/timeseries.py:26: UserWarning: It is recommended to set 'order_channel_by_depth' to True when using the sortingview backend
  warnings.warn(


Could not load font to use in scalebar. Scalebar will not be drawn.
https://figurl.org/f?v=gs://figurl/spikesortingview-10&d=sha1://e5b09329364c0c918c0c819d17ad964cb8568371&label=SpikeInterface%20-%20Timeseries


FigurlFigure(data_uri='sha1://79f6a37036e87f3417f489c0b8998868bd0dcd0a', height=500, view_uri='gs://figurl/spi…

https://figurl.org/f?v=gs://figurl/spikesortingview-10&d=sha1://79f6a37036e87f3417f489c0b8998868bd0dcd0a&label=SpikeInterface%20-%20Sorting%20Summary


In [16]:
v_summary = vv.TabLayout(
                items=[
                    vv.TabLayoutItem(
                        label='Sorting Summary',
                        view=we_ss.view
                    ),
                    vv.TabLayoutItem(
                        label='Timeseries',
                        view=we_ts.view
                    ),
                ]
            )
# generate URL
url = v_summary.url(label="SpikeInterface - Sorting Summary")
url

'https://figurl.org/f?v=gs://figurl/spikesortingview-10&d=sha1://bf49e0983e8936a32caccc0a87829a39f6151122&label=SpikeInterface%20-%20Sorting%20Summary'

Apply curation to sorting and export

In [ ]:
sorting_curated = scur.apply_sortingview_curation(sorting, '/Users/jcheng/Downloads/sorting-curation.json', exclude_labels=["reject"])